# Biber Resultate

- Welche Klasse erreichte durchschnittlich die meisten/wenigsten Punkte
- Welche Aufgabe wurde am besten/schlechtesten gelöst?
- Welche Punkte wurden wie häufig gegeben?
- Wie entwicklte sich die Punktezahl der gesamten Schule über die Jahre?
- Wie entwicklte sich die Teilnehmeranzahl der gesamten Schule über die Jahre?

## Daten einlesen

In [ ]:
import pandas as pd

In [ ]:
%%bash
tree

In [ ]:
data_2024 = pd.read_csv("data/2024.csv")
data_2025a = pd.read_csv("data/2025a.csv")
data_2025b = pd.read_csv("data/2025b.csv")

In [ ]:
data_2024

In [ ]:
data_2025a

In [ ]:
data_2025b

## Daten zusammenführen

In den verschiedenen Jahren gibt es unterschiedliche Aufgaben. Wir überführen die Daten deshalb in ein anderes Format - das sogenannte Long-Format - so dass alle Jahre in den selben Dataframe gespeichert und dann gemeinsam analysiert werden können.

|Jahr|Klassenstufe|Klasse|Aufgabe|Punkte|
|--|--|--|--|--|
|2024|Schuljahre 11, 12 und 13|KHR|Happy Birthday|-2|
|...|...|...|...|...|

In [ ]:
data_2024_long = pd.melt(
    data_2024,
    id_vars=['Jahr', 'Klassenstufe', 'Klasse'],     # columns that stay as identifiers
    var_name='Aufgabe',   # name for the new "variable" column
    value_name='Punkte'     # name for the new "value" column
)

In [ ]:
data_2024_long

In [ ]:
data_2025a_long = pd.melt(
    data_2025a,
    id_vars=['Jahr', 'Klassenstufe', 'Klasse'],
    var_name='Aufgabe',
    value_name='Punkte'
)

In [ ]:
data_2025a_long

In [ ]:
data_2025b_long = pd.melt(
    data_2025b,
    id_vars=['Jahr', 'Klassenstufe', 'Klasse'],
    var_name='Aufgabe',
    value_name='Punkte'
)

In [ ]:
data_2025b_long

Jetzt müssen wir die drei Dataframes für die Analyse zusammenführen. In diesem Fall also einfach untereinander schreiben, da sie die gleichen Spaltennamen haben.

In [ ]:
df = pd.concat([data_2024_long, data_2025a_long, data_2025b_long])

In [ ]:
df

Wir entfernen die Bonuspunkte. Diese Punkte bekommt jede(r) Teilnehmer(in) und sie dienen nur dazu, dass kein Teilnehmer weniger als 0 Punkte erreichen kann. Wir benötigen diese nicht für die Analyse - im Gegenteil, die Bonuspunkte stören sogar.

In [ ]:
df = df[df["Aufgabe"] != "Bonus"]

## Daten analysieren

In [ ]:
df.describe()

### Welche Klasse erreichte durchschnittlich die meisten/wenigsten Punkte

In [ ]:
df_avg = df.groupby(["Jahr", "Klassenstufe", "Klasse"])["Punkte"].mean()

In [ ]:
df_avg.sort_values()

In [ ]:
df.boxplot(column='Punkte', by='Klasse', showmeans=True, grid=False)

### Welche Aufgabe wurde am besten/schlechtesten gelöst?

In [ ]:
df_avg = df.groupby(["Aufgabe"])["Punkte"].mean()

In [ ]:
df_avg.sort_values()

In [ ]:
df.boxplot(column="Punkte", by="Aufgabe", grid=False, showmeans=True, rot=90)

### Wie entwickelt sich die durchschnittliche Punktezahl der Schule über die Jahre?

In [ ]:
df_avg = (
    df
    .groupby(["Jahr"])["Punkte"]
    .mean()
)

In [ ]:
df_avg.sort_values()

In [ ]:
df.boxplot(column='Punkte', by='Jahr', showmeans=True, grid=False)

### Wie entwicklte sich die Teilnehmeranzahl der gesamten Schule über die Jahre?

Dazu verwenden wir - der Einfachheit halber - einfach die Originaldaten. Dort wurde pro Zeile ein Teilnehmer aufgelistet. Wir müssen also einfach nur die Zeilen zählen.

In [ ]:
teilnehmer = {
    "Jahr": [len(data_2024), len(data_2025a) + len(data_2025b)],
}
new_df = pd.DataFrame(teilnehmer, index=["2024", "2025"])

In [ ]:
new_df

In [ ]:
new_df.plot.bar()

### Welche Punkte wurden wie häufig gegeben?

In [ ]:
df[["Punkte"]].hist()

## Anhang

### Excel to csv

In [ ]:
excel_file = "data/Biber.xlsx"
data_2024 = pd.read_excel(excel_file, sheet_name="2024")
data_2025a = pd.read_excel(excel_file, sheet_name="2025_09_10")
data_2025b = pd.read_excel(excel_file, sheet_name="2025_11_12_13")

In [ ]:
data_2024.to_csv("data/2024.csv", index=False)
data_2025a.to_csv("data/2025a.csv", index=False)
data_2025b.to_csv("data/2025b.csv", index=False)

Damit wir in Excel mit Pivot-Table arbeiten können, speichern wir die Long-Format-Datei ab.

In [ ]:
df.to_csv("data/Biber-Daten-Long.csv", index=False)

### Bilder als Dateien abspeichern

In [ ]:
img = df.boxplot(column='Punkte', by='Jahr', showmeans=True, grid=False)
img.get_figure().savefig('img/a_boxplot.svg')
img.get_figure().savefig('img/a_boxplot.png')